<a href="https://colab.research.google.com/github/v-grand/Collector/blob/main/Crypto_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal

In [ ]:
!pip install ta
!pip install mplfinance

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=aead026da6e278582b1676745bbc98be13debe4c41bf3ddbd2f0ba0e54ee42bd
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.7 MB/s eta 0:00:00


### Import

In [ ]:

'''
%pip install prophet

'''
import requests
import urllib

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

import pandas as pd
from prophet import Prophet

from datetime import datetime, timedelta
import numpy as np
from pandas import DataFrame
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
import ta
from typing import Tuple
import yfinance as yf

from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error


from google.colab import drive
#from tensorflow.keras.models import load_model

from prophet import Prophet

# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Function

In [1]:
def prep_data(drop_na:bool = True, ticker='BTC-USD', period='60d', interval_='5m') -> DataFrame:

    N_pred=3

    data = yf.download(ticker, period=period, interval=interval_)


    data.drop(columns=['Adj Close','Volume'], inplace=True)

    data['NextOpen'] = data['Open'].shift(-1)
    data['NextHigh'] = data['High'].shift(-1)
    data['NextLow'] = data['Low'].shift(-1)
    data['NextClose'] = data['Close'].shift(-1)


    for i in range(1, N_pred):
      data[f'PrevOpen_bar{i}'] = data['Open'].shift(i)
      data[f'PrevHigh_bar{i}'] = data['High'].shift(i)
      data[f'PrevLow_bar{i}'] = data['Low'].shift(i)
      data[f'PrevClose_bar{i}'] = data['Close'].shift(i)

    data = data.iloc[N_pred:]


    # Drop NaN values
    if(drop_na):
        data.dropna(inplace=True)

    data.reset_index(inplace=True)

    if data.empty:
      print(f'No data found for {ticker}')
      return None
    else:

      if 'Date' in data.columns:
        last_date_value= data['Date'].iloc[-1]
        first_data_value=data['Date'].iloc[0]
        print(f'   {ticker} load at {first_data_value} to {last_date_value} interval {interval_}')

      elif 'Datetime' in data.columns:
        last_date_value= data['Datetime'].iloc[-1]
        first_data_value=data['Datetime'].iloc[0]
        print(f'   {ticker} load at {first_data_value} to {last_date_value} interval {interval_}')

    data.rename(columns=lambda x: f'{x}_{interval_}', inplace=True)

    return data


NameError: name 'DataFrame' is not defined

In [2]:
def scale_data(data:DataFrame, columns_x, columns_y) -> Tuple[MinMaxScaler, np.ndarray, np.ndarray]:

    # Prepare target variable
    y = data[columns_y].values
    X = data [columns_x]

    # Scale features
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).reshape(-1)

    return scaler, X_scaled, y_scaled

NameError: name 'DataFrame' is not defined

#3mo

In [ ]:
interval_ = '3mo'
period_ = 'max'  # или другой поддерживаемый период, например '10y' для последних 10 лет

ticker_3mo = 'XRP-USD'
data_3mo = prep_data(drop_na=True, ticker=ticker_3mo, period=period_, interval_=interval_)

# Проверяем, что данные загружены корректно
if data_3mo is not None and not data_3mo.empty:
    # Последняя строка
    last_row = data_3mo.iloc[-1]
    print(last_row)
else:
    print("No data found for", ticker_3mo)


[*********************100%***********************]  1 of 1 completed

   XRP-USD load at 2018-08-01 00:00:00+00:00 to 2024-08-01 00:00:00+00:00 interval 3mo
Price               Ticker     
Date_3mo            _3mo           2024-08-01 00:00:00+00:00
Close_3mo           XRP-USD_3mo                     0.509237
High_3mo            XRP-USD_3mo                     0.662206
Low_3mo             XRP-USD_3mo                     0.433492
Open_3mo            XRP-USD_3mo                     0.623934
NextOpen_3mo        _3mo                            0.509237
NextHigh_3mo        _3mo                            0.541252
NextLow_3mo         _3mo                            0.493652
NextClose_3mo       _3mo                            0.535262
PrevOpen_bar1_3mo   _3mo                            0.500062
PrevHigh_bar1_3mo   _3mo                            0.657911
PrevLow_bar1_3mo    _3mo                            0.391139
PrevClose_bar1_3mo  _3mo                            0.623939
PrevOpen_bar2_3mo   _3mo                            0.503176
PrevHigh_bar2_3mo   _3mo   

In [ ]:
columns_x = ['Open_3mo', 'High_3mo', 'Low_3mo', 'Close_3mo',
           'PrevOpen_bar1_3mo', 'PrevHigh_bar1_3mo', 'PrevLow_bar1_3mo',
           'PrevClose_bar1_3mo', 'PrevOpen_bar2_3mo', 'PrevHigh_bar2_3mo',
           'PrevLow_bar2_3mo', 'PrevClose_bar2_3mo']
columns_y = ['NextClose_3mo']
# ['NextOpen_3mo', 'NextHigh_3mo', 'NextLow_3mo', 'NextClose_3mo']
scaler, X_scaled, y_scaled = scale_data(data_3mo, columns_x, columns_y)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

model_name = "/content/drive/My Drive/High_random_forest.joblib"
    # Random Forest model
model_High = RandomForestRegressor(n_estimators=100, random_state=42)
model_High.fit(X_train, y_train)

    # Save the model to Google Drive
joblib.dump(model_High, model_name)

    # Predict the prices
predicted_prices = model_High.predict(X_test)

    # Evaluate
mse = mean_squared_error(y_test, predicted_prices)
print(f'Mean Squared Error: {mse}')
#feature_importance_high = model_High.feature_importances_
#feature_df_high = pd.DataFrame({'Feature': data_x.columns, 'Importance': feature_importance_high})

In [ ]:
# developer: Foundry Digital
# Copyright © 2023 Foundry Digital

# Import necessary modules to use for model creation - can be downloaded using pip
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def create_and_save_base_model_lstm(scaler:MinMaxScaler, X_scaled:np.ndarray, y_scaled:np.ndarray) -> float:
    """
    Base model that can be created for predicting the S&P 500 close price

    The function creates a base model, given a scaler, inputs and outputs, and
    stores the model weights as a .h5 file in the mining_models/ folder. The model
    architecture and model name given now is a placeholder, can (and should)
    be changed by miners to build more robust models.

    Input:
        :param scaler: The scaler used to scale the inputs during model training process
        :type scaler: sklearn.preprocessing.MinMaxScaler

        :param X_scaled: The already scaled input data that will be used by the model to train and test
        :type X_scaled: np.ndarray

        :param y_scaled: The already scaled output data that will be used by the model to train and test
        :type y_scaled: np.ndarray

    Output:
        :returns: The MSE of the model on the test data
        :rtype: float
    """
    model_name = "mining_models/base_lstm"

    # Reshape input for LSTM
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # LSTM model - all hyperparameters are baseline params - should be changed according to your required
    # architecture. LSTMs are also not the only way to do this, can be done using any algo deemed fit by
    # the creators of the miner.
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_scaled.shape[1], X_scaled.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32)
    model.save(f'{model_name}.h5')

    # Predict the prices - this is just for a local test, this prediction just allows
    # miners to assess the performance of their models on real data.
    predicted_prices = model.predict(X_test)

    # Rescale back to original range
    predicted_prices = scaler.inverse_transform(predicted_prices)
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate
    mse = mean_squared_error(y_test_rescaled, predicted_prices)
    print(f'Mean Squared Error: {mse}')

    return mse

def create_and_save_base_model_regression(scaler:MinMaxScaler, X_scaled:np.ndarray, y_scaled:np.ndarray) -> float:
    """
    Base model that can be created for predicting the S&P 500 close price

    The function creates a base model, given a scaler, inputs and outputs, and
    stores the model weights as a .h5 file in the mining_models/ folder. The model
    architecture and model name given now is a placeholder, can (and should)
    be changed by miners to build more robust models.

    Input:
        :param scaler: The scaler used to scale the inputs during model training process
        :type scaler: sklearn.preprocessing.MinMaxScaler

        :param X_scaled: The already scaled input data that will be used by the model to train and test
        :type X_scaled: np.ndarray

        :param y_scaled: The already scaled output data that will be used by the model to train and test
        :type y_scaled: np.ndarray

    Output:
        :returns: The MSE of the model on the test data
        :rtype: float
    """
    model_name = "mining_models/base_linear_regression"

    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # LSTM model - all hyperparameters are baseline params - should be changed according to your required
    # architecture. LSTMs are also not the only way to do this, can be done using any algo deemed fit by
    # the creators of the miner.
    model = LinearRegression()
    model.fit(X_train, y_train)

    '''with h5py.File(f'{model_name}.h5', 'w') as hf:
        hf.create_dataset('coefficients', data=model.coef_)
        hf.create_dataset('intercept', data=model.intercept_)'''
    joblib.dump(model, f"{model_name}.joblib")

    # Predict the prices - this is just for a local test, this prediction just allows
    # miners to assess the performance of their models on real data.
    predicted_prices = model.predict(X_test)

    # Rescale back to original range
    predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate
    mse = mean_squared_error(y_test_rescaled, predicted_prices)
    print(f'Mean Squared Error: {mse}')

    return mse

# Spread

In [ ]:
interval_='1d'
period_='300d'

ticker_2='XRP-USD'
ticker_1='ATOM-USD'
K_1=1
K_2=20

data_1 = prep_data(drop_na=True, ticker=ticker_1, period=period_, interval_=interval_)
#print(data_1.shape)

data_2 = prep_data(drop_na=True, ticker=ticker_2, period=period_, interval_=interval_)
#print(data_2.shape)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

   ATOM-USD load at 2019-03-17 00:00:00 to 2025-01-02 00:00:00 interval 1d
   XRP-USD load at 2017-11-12 00:00:00 to 2025-01-02 00:00:00 interval 1d


In [ ]:
# Объединяем датасеты по столбцу Datetime
combined_df = pd.merge(data_1, data_2, on='Datetime_1h', how='inner')

combined_df['Close']=K_1*combined_df['Close_1h_x']-K_2*combined_df['Close_1h_y']
#combined_df.info()

TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

In [ ]:
from prophet import Prophet

dp_ds = combined_df[['Datetime_1h', 'Close']].copy()
dp_ds['Datetime_1h'] = dp_ds['Datetime_1h'].dt.tz_localize(None)

# Подготовка данных для Prophet
dp_ds = dp_ds.rename(columns={'Datetime_1h': 'ds', 'Close': 'y'})


model = Prophet(
    growth='linear',
    changepoint_range=0.8,
    seasonality_mode='additive',
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    changepoint_prior_scale=0.05,
    interval_width=0.8,
    uncertainty_samples=10
)
# Продолжаем обучение и прогнозирование

model.fit(dp_ds)

# Создание будущего DataFrame для прогнозов
future = model.make_future_dataframe(periods=30)  # Прогноз на 30 дней вперед

# Получение прогнозов
forecast = model.predict(future)



In [ ]:
# Построение графика прогнозов
fig = model.plot( forecast.iloc[-100:])
plt.xlabel(f'Date')
plt.ylabel('Quote')
plt.title(f'{ticker_1}x{K_1} and {ticker_2}x{K_2}')
plt.show()